# V1
ETL
First Version

In [7]:
import pandas as pd
import csv

# Load the Excel file with xlrd engine for .xls files
excel_file = r'./Estructura HSN.sept24.xls'
df = pd.read_excel(excel_file, engine='xlrd')

# Skip the first row (index 0)
df = df.iloc[1:]

# Open the CSV file for writing
csv_file = 'outputV1.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['ID', 'parentID', 'officename', 'official', 'currentRegulations'])
    
    # Define ID counter and parentID
    id_counter = 100
    last_id_per_column = {}  # Dictionary to store the last used ID for each column
    
    # Iterate over each column A to H
    for col_index, col_letter in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], start=1):
        # Reset parentID when moving to the next column
        parentID = None
        
        # Go row by row in each column
        for row_index, value in enumerate(df.iloc[:, col_index - 1]):
            if pd.notna(value):  # If there's a value in the current cell
                # Assign values to the row based on the column content
                officename = value.replace('"', '').replace(',', ' ')  # Remove internal quotes

                official = df.iloc[row_index, 8] if pd.notna(df.iloc[row_index, 8]) else None  # Column I
                currentRegulations = df.iloc[row_index, 9] if pd.notna(df.iloc[row_index, 9]) else None  # Column J
                
                # Determine parentID logic
                if col_index == 0:  # Column A (first column)
                    parentID = None  # Top-level, no parent
                else:
                    parentID = last_id_per_column.get(col_index - 1, None)  # Get the last ID from the previous column
                
                # Write the data into the CSV
                writer.writerow([id_counter, parentID, officename, official, currentRegulations])
                
                # Update the last ID used for this column
                last_id_per_column[col_index] = id_counter
                
                # Increment the ID counter
                id_counter += 1
               

# V2 - V3 - V4 - V5 

Lost because i m a rookie

# VF

ETL
V6 - Final Version 

In [6]:
import pandas as pd
import csv

# Load the Excel file
excel_file = './Estructura HSN.sept24.xls'
df = pd.read_excel(excel_file, engine='xlrd')

# Open the CSV file for writing
csv_file = 'outputVF.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['id', 'parentId', 'hierarchies', 'officename', 'official', 'currentRegulations'])
    
    # Define ID counter and parentID
    id_counter = 100  # Start ID from 100
    last_id_per_column = {}  # Dictionary to store the last used ID for each column index
    last_column_index = -1  # Initialize with a value less than any valid column index
    
    # Iterate over each row
    for row_index in range(len(df)):
        parentId = None  # Reset parentID for each row
        
        # Go column by column, from H (last column) to A (first column)
        for col_index in range(7, -1, -1):  # 7 = column H, 0 = column A
            value = df.iloc[row_index, col_index]
            
            if pd.notna(value):  # If there's a value in the current cell
                officename = value.replace('"', '').replace(',', ' ')  # Clean the officename
                
                official = df.iloc[row_index, 8] if pd.notna(df.iloc[row_index, 8]) else None  # Column I
                currentRegulations = df.iloc[row_index, 9] if pd.notna(df.iloc[row_index, 9]) else None  # Column J
                
                # Determine parentID based on column index comparison
                if col_index > last_column_index:
                    parentId = last_id_per_column.get(last_column_index, None)
                    
                elif col_index == last_column_index:
                    # If column index is equal, search for parentID from a previous column
                    for check_index in range(col_index - 1, -1, -1):
                        if check_index in last_id_per_column:
                            parentId = last_id_per_column[check_index]
                            break

                elif col_index < last_column_index:
                    # If column index is smaller, look backward for a column index that is smaller than the current col_index
                    for check_index in range(col_index - 1, -1, -1):  # Search backward until the first column (A)
                        if check_index in last_id_per_column:
                            parentId = last_id_per_column[check_index]
                            break
                
                # Write the data into the CSV
                writer.writerow([id_counter, parentId, chr(col_index + 65), officename, official, currentRegulations])
                
                # Update the last ID used for this column
                last_id_per_column[col_index] = id_counter
                
                # Update last_column_index to the current column index
                last_column_index = col_index
                
                # Increment the ID counter
                id_counter += 1